In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,gc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve,auc,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import time
plt.style.use('seaborn-whitegrid')
import warnings
warnings.simplefilter("ignore")
colors = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

In [3]:
train_trn = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")

In [4]:
train_id = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")

In [5]:
df_train = train_trn.merge(train_id,on=['TransactionID'],how='left')
#print(f'Shape of identity train data: {train_id.shape}')

In [6]:
del train_trn
del train_id

In [7]:
df_test_transaction = pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv')
df_test_identity = pd.read_csv('../input/ieee-fraud-detection/test_identity.csv')
print(f'Shape of transaction train data: {df_test_transaction.shape}')
print(f'Shape of identity train data: {df_test_identity.shape}')

Shape of transaction train data: (506691, 393)
Shape of identity train data: (141907, 41)


In [8]:
# #combining frames
# # if some columns (id) are wrongly labeled in test_identity. We will correct it
id_cols = [col for col in df_test_identity.columns if col[0]+col[1] == 'id']
rename_cols = {i:'id_'+str(i[-2]+i[-1]) for i in id_cols}
df_test_identity = df_test_identity.rename(columns=rename_cols)
df_test = df_test_transaction.merge(df_test_identity,on=['TransactionID'],how='left')
del df_test_transaction
del df_test_identity


In [9]:
#columns with different types

obj_col = df_train.select_dtypes(include='O').columns
float_col = df_train.select_dtypes(include='float64').columns
int_col = df_train.select_dtypes(include='int64').columns

In [10]:
df_train[float_col] = df_train[float_col].astype('float32')

In [11]:
#df_train = pd.read_csv('data/train_combined.csv')
#df_test = pd.read_csv('data/test_combined.csv')
print(df_train.shape)
print(df_test.shape)

(590540, 434)
(506691, 433)


MISSING VALUES

In [12]:
 top_missing_cols(
defdf,n=45,thresh=80):
    """
    returns missing columns in dataframe with missing values percent > thresh
    if n=None. It will gave whole dataframe with missing values percent > thresh
    """
    
    dff = (df.isnull().sum()/df.shape[0])*100
    dff = dff.reset_index()
    dff.columns = ['col','missing_percent']
    dff = dff.sort_values(by=['missing_percent'],ascending=False).reset_index(drop=True)
    print(f'There are {df.isnull().any().sum()} columns in this dataset with missing values.')
    print(f'There are {dff[dff["missing_percent"] > thresh].shape[0]} columns with missing percent values than {thresh}%')
    if n:
        return dff.head(n)
    else:
        return dff

SyntaxError: invalid syntax (<ipython-input-12-2ade9a2d0c6c>, line 2)

In [13]:
top_missing_cols(df_train, thresh=80)

NameError: name 'top_missing_cols' is not defined

There are 385 columns in this dataset with missing values.
There are 41 columns with missing percent values than 90%

In [14]:
#Remove variables with missing values more than 30 percent

A = (np.sum(pd.isnull(df_train)).sort_values(ascending=False)/len(df_train))*100
Removed_col = A[A>0.8].index
df_train.drop(columns=Removed_col, inplace=True)

In [15]:
A = (np.sum(pd.isnull(df_test)).sort_values(ascending=False)/len(df_train))*100
Removed_col = A[A>0.8].index
df_test.drop(columns=Removed_col, inplace=True)

In [16]:
#fig, axes = plt.subplots(1,2,figsize=(15,5))

#sns.scatterplot(y=df_train['TransactionAmt'],x=df_train['TransactionDT'],hue=df_train['isFraud'],ax=axes[0])
#axes[0].title.set_text('Transcation Amount - Train')

#sns.scatterplot(y=df_test['TransactionAmt'],x=df_test['TransactionDT'],ax=axes[1])
#axes[1].title.set_text('Transcation Amount - Test')

#plt.show()

In [17]:
#h = df_train[df_train["TransactionAmt"]>30000].index
#df_train[df_train["TransactionAmt"]>30000]

In [18]:
#h = df_train[df_train["TransactionAmt"]>30000].index
#h = df_train.set_index("TransactionID")
#df_train = df_train.drop([3261336,3261339],axis = 0)#, inplace = True)

In [19]:
 df_train = df_train[df_train['TransactionAmt'] < 30000]


In [20]:
train_dummy = pd.get_dummies(df_train, drop_first = True)

In [21]:
train_dummy.drop(columns = "TransactionID", inplace = True)

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_dummy.drop(columns='isFraud'), train_dummy['isFraud'], test_size=0.25, random_state=0)



params = {'num_leaves': 260,
          'min_child_samples': 80,
          'objective': 'binary',
          'max_depth': 15,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          'is_unbalance':True, # unbalanced dataset 3.5%

         }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid)
    
bst = lgb.train(params, d_train, num_boost_round= 400, valid_sets=[d_train, d_valid], verbose_eval=50, early_stopping_rounds=50)

Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.95836	valid_1's auc: 0.925051
[100]	training's auc: 0.96769	valid_1's auc: 0.932383
[150]	training's auc: 0.974988	valid_1's auc: 0.938258
[200]	training's auc: 0.979859	valid_1's auc: 0.942381
[250]	training's auc: 0.983835	valid_1's auc: 0.944935
[300]	training's auc: 0.986739	valid_1's auc: 0.946232
[350]	training's auc: 0.989372	valid_1's auc: 0.948214
[400]	training's auc: 0.991288	valid_1's auc: 0.949622
Did not meet early stopping. Best iteration is:
[400]	training's auc: 0.991288	valid_1's auc: 0.949622


In [24]:
test_dummy = pd.get_dummies(df_test, drop_first=True)
#test_dummy.info()
float_col = test_dummy.select_dtypes(include='float64').columns
test_dummy[float_col] = test_dummy[float_col].astype("float32")

In [25]:
train_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590538 entries, 0 to 590539
Columns: 117 entries, isFraud to card6_debit or credit
dtypes: float32(104), int64(3), uint8(10)
memory usage: 257.9 MB


In [26]:
y_pred = bst.predict(test_dummy,predict_disable_shape_check=True)

In [27]:
y_pred 

array([0.02665071, 0.03591207, 0.05809389, ..., 0.06039619, 0.06039619,
       0.01409208])

In [28]:
submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
submission['isFraud'] = y_pred
submission.to_csv('submission.csv',index=False)
submission.head()

,TransactionID,isFraud
0,3663549,0.026651
1,3663550,0.035912
2,3663551,0.058094
3,3663552,0.042093
4,3663553,0.061020
